In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
# Mount Google Drive and set data paths.
import os
from google.colab import drive
from google.colab import files

drive.mount('/content/gdrive')

Mounted at /content/gdrive


# ML4H - Project 2

## Preprocessing

### General

In [ ]:
# Gloabal variables
data_path = "/content/gdrive/My Drive/ETH/MLH/Project2/data/PubMed_200k_RCT/"

seed = 50

In [ ]:
import pickle
import numpy as np
import string

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
from nltk.corpus import stopwords

from tqdm.notebook import tqdm

import pandas as pd

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

### Reading files

In [ ]:
with open(data_path + 'train.txt') as f:
    train = f.readlines()

with open(data_path + 'test.txt') as f:
    test = f.readlines()

with open(data_path + 'dev.txt') as f:
    dev = f.readlines()

print("Train has size: {}".format(len(train)))
print("Test has size: {}".format(len(test)))
print("Dev has size: {}".format(len(dev)))

Train has size: 2593169
Test has size: 34493
Dev has size: 33932


In [ ]:
# Removing ID and empty lines
train = list(filter(lambda x: not x.startswith(('###', '\n')), tqdm(train)))
test = list(filter(lambda x: not x.startswith(('###', '\n')), tqdm(test)))
dev = list(filter(lambda x: not x.startswith(('###', '\n')), tqdm(dev)))

  0%|          | 0/2593169 [00:00<?, ?it/s]

  0%|          | 0/34493 [00:00<?, ?it/s]

  0%|          | 0/33932 [00:00<?, ?it/s]

In [ ]:
# Building list of lists (and separating first word)
train = list(map(lambda x:x.split('\t', 1), tqdm(train)))
test = list(map(lambda x:x.split('\t', 1), tqdm(test)))
dev = list(map(lambda x:x.split('\t', 1), tqdm(dev)))

  0%|          | 0/2211861 [00:00<?, ?it/s]

  0%|          | 0/29493 [00:00<?, ?it/s]

  0%|          | 0/28932 [00:00<?, ?it/s]

In [ ]:
print(train[0])

['BACKGROUND', 'The emergence of HIV as a chronic condition means that people living with HIV are required to take more responsibility for the self-management of their condition , including making physical , emotional and social adjustments .\n']


In [ ]:
# Lowercasing sentence
train = list(map(lambda x:[x[0], x[1].lower()], tqdm(train)))
test = list(map(lambda x:[x[0], x[1].lower()], tqdm(test)))
dev = list(map(lambda x:[x[0], x[1].lower()], tqdm(dev)))

  0%|          | 0/2211861 [00:00<?, ?it/s]

  0%|          | 0/29493 [00:00<?, ?it/s]

  0%|          | 0/28932 [00:00<?, ?it/s]

In [ ]:
print(train[0])

['BACKGROUND', 'the emergence of hiv as a chronic condition means that people living with hiv are required to take more responsibility for the self-management of their condition , including making physical , emotional and social adjustments .\n']


In [ ]:
# Tokenizing sentence
tokenizer = RegexpTokenizer(r'\w+')

train = list(map(lambda x:[x[0], tokenizer.tokenize(x[1])], tqdm(train)))
test = list(map(lambda x:[x[0], tokenizer.tokenize(x[1])], tqdm(test)))
dev = list(map(lambda x:[x[0], tokenizer.tokenize(x[1])], tqdm(dev)))

  0%|          | 0/2211861 [00:00<?, ?it/s]

  0%|          | 0/29493 [00:00<?, ?it/s]

  0%|          | 0/28932 [00:00<?, ?it/s]

In [ ]:
print(train[0])

['BACKGROUND', ['the', 'emergence', 'of', 'hiv', 'as', 'a', 'chronic', 'condition', 'means', 'that', 'people', 'living', 'with', 'hiv', 'are', 'required', 'to', 'take', 'more', 'responsibility', 'for', 'the', 'self', 'management', 'of', 'their', 'condition', 'including', 'making', 'physical', 'emotional', 'and', 'social', 'adjustments']]


In [ ]:
# Removing stopwords
stopwords_set = stopwords.words('english')

train = list(map(lambda x:[x[0], [w for w in x[1] if not w in stopwords_set]], tqdm(train)))
test = list(map(lambda x:[x[0], [w for w in x[1] if not w in stopwords_set]], tqdm(test)))
dev = list(map(lambda x:[x[0], [w for w in x[1] if not w in stopwords_set]], tqdm(dev)))

  0%|          | 0/2211861 [00:00<?, ?it/s]

  0%|          | 0/29493 [00:00<?, ?it/s]

  0%|          | 0/28932 [00:00<?, ?it/s]

In [ ]:
print(train[0])

['BACKGROUND', ['emergence', 'hiv', 'chronic', 'condition', 'means', 'people', 'living', 'hiv', 'required', 'take', 'responsibility', 'self', 'management', 'condition', 'including', 'making', 'physical', 'emotional', 'social', 'adjustments']]


In [ ]:
# Stemming
stemmer = PorterStemmer()

train = list(map(lambda x:[x[0], [stemmer.stem(w) for w in x[1]]], tqdm(train)))
test = list(map(lambda x:[x[0], [stemmer.stem(w) for w in x[1]]], tqdm(test)))
dev = list(map(lambda x:[x[0], [stemmer.stem(w) for w in x[1]]], tqdm(dev)))

  0%|          | 0/2211861 [00:00<?, ?it/s]

  0%|          | 0/29493 [00:00<?, ?it/s]

  0%|          | 0/28932 [00:00<?, ?it/s]

In [ ]:
print(train[0])

['BACKGROUND', ['emerg', 'hiv', 'chronic', 'condit', 'mean', 'peopl', 'live', 'hiv', 'requir', 'take', 'respons', 'self', 'manag', 'condit', 'includ', 'make', 'physic', 'emot', 'social', 'adjust']]


In [ ]:
# Lemmatizing
lemmatizer = WordNetLemmatizer()

train = list(map(lambda x:[x[0], [lemmatizer.lemmatize(w) for w in x[1]]], tqdm(train)))
test = list(map(lambda x:[x[0], [lemmatizer.lemmatize(w) for w in x[1]]], tqdm(test)))
dev = list(map(lambda x:[x[0], [lemmatizer.lemmatize(w) for w in x[1]]], tqdm(dev)))

  0%|          | 0/2211861 [00:00<?, ?it/s]

  0%|          | 0/29493 [00:00<?, ?it/s]

  0%|          | 0/28932 [00:00<?, ?it/s]

In [ ]:
print(train[0])

['BACKGROUND', ['emerg', 'hiv', 'chronic', 'condit', 'mean', 'peopl', 'live', 'hiv', 'requir', 'take', 'respons', 'self', 'manag', 'condit', 'includ', 'make', 'physic', 'emot', 'social', 'adjust']]


In [ ]:
# N Grams
N = 1

if N > 1:
  train = list(map(lambda x:[x[0], train + [ ' '.join(grams) for grams in ngrams(x[1], N)]], tqdm(train)))
  test = list(map(lambda x:[x[0], test + [ ' '.join(grams) for grams in ngrams(x[1], N)]], tqdm(test)))
  dev = list(map(lambda x:[x[0], dev + [ ' '.join(grams) for grams in ngrams(x[1], N)]], tqdm(dev)))

In [ ]:
print(train[0])

['BACKGROUND', ['emerg', 'hiv', 'chronic', 'condit', 'mean', 'peopl', 'live', 'hiv', 'requir', 'take', 'respons', 'self', 'manag', 'condit', 'includ', 'make', 'physic', 'emot', 'social', 'adjust']]


In [ ]:
train_df = pd.DataFrame(train, columns=['Label', 'Words'])
test_df = pd.DataFrame(test, columns=['Label', 'Words'])
dev_df = pd.DataFrame(dev, columns=['Label', 'Words'])

In [ ]:
# Save pre processed
train_df.to_parquet(data_path + 'pre_processed_train.gzip', compression='gzip')
test_df.to_parquet(data_path + 'pre_processed_test.gzip', compression='gzip')
dev_df.to_parquet(data_path + 'pre_processed_dev.gzip', compression='gzip')

In [ ]:
train_df.head(5)

,Label,Words
0,BACKGROUND,"[emerg, hiv, chronic, condit, mean, peopl, liv..."
1,BACKGROUND,"[paper, describ, design, evalu, posit, outlook..."
2,METHODS,"[studi, design, randomis, control, trial, men,..."
3,METHODS,"[intervent, group, particip, onlin, group, pro..."
4,METHODS,"[program, base, self, efficaci, theori, use, s..."
